In [6]:
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import os
os.environ['PYTHONWARNINGS'] = "ignore"
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import pickle
import xgboost as xgb
from shap import GPUTreeExplainer
from matplotlib.ticker import MaxNLocator,MultipleLocator
from scipy.stats import kendalltau
from scipy.special import expit

import mlresearch
mlresearch.utils.set_matplotlib_style()
from mlresearch.utils import set_matplotlib_style
set_matplotlib_style(font_size=27)

## Preprocessing

In [7]:
# Use White alone & African American only 
FEAT_CNT = 8
STATE = 'VA'
FOLDS = 5
seeds = [0,21,42,63,84]

In [8]:
import numpy as np
print(np.__version__) #1.26.4
# print(shap.__version__) #0.46.1.dev86
print(sklearn.__version__) #1.6.0
print(xgb.__version__) #1.7.6
import sys
print(sys.version)

2.0.2
1.6.1
2.1.4
3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:56:27) [GCC 11.2.0]


In [9]:
categorical_cols =['Occupation', 'Marriage','Place of Birth','Sex', 'Race']

with open(file=f'dataset/ACS_Income_{STATE}.pickle', mode='rb') as f:
    df=pickle.load(f)
columns = df.columns
with pd.option_context('future.no_silent_downcasting', True):
    df.replace([' <=50K',' >50K'],
                 [0,1], inplace = True)
    df['Sex'].replace( {'Female':0.0},inplace = True)
    df['Sex'].replace({'Male':1.0}, inplace = True)
df['Race'] = df['Race'].str.strip().str.lower()
race_used = ['asian alone', 'black or african american alone', 'white alone']

# Replace values not in race_used with 'Other'
df['Race'] = df['Race'].apply(lambda x: x if x in race_used else 'other')
X = df.iloc[:, 0:FEAT_CNT]
Y = df.iloc[:, FEAT_CNT]

category_col =['Occupation', 'Marriage','Place of Birth', 'Race']
X = pd.get_dummies(X, columns=category_col, drop_first=False)
for c in X.columns:
    X[c] = X[c].astype(float)

In [10]:
df['Race'].unique()

array(['black or african american alone', 'white alone', 'other',
       'asian alone'], dtype=object)

In [11]:
'''
2 buckets
    White, Black + Asian + Other
    White + Black , Asian + Other ( Number-wise majority vs minority)
    White + Asian, Black + Other (Privileged vs unprivileged)
3 buckets
    White, Black, Asian + Other
    White, Asian, Black + Other
'''

white, black, asian, other =  'Race_white alone','Race_black or african american alone', 'Race_asian alone', 'Race_other'
all_races = [white, black, asian,other]

## Utils

In [126]:
def assign_race(X, target_race):
    X2 = X.copy()
    # 1) 먼저 모든 race 컬럼을 0으로 초기화
    for r in all_races:
        X2[r] = 0.0

    # 2) 원본에서 target_race==1 이었던 행들은 다시 1로 돌려놓기
    mask_target = X[target_race] == 1
    X2.loc[mask_target, target_race] = 1.0

    # 3) target_race가 아닌 나머지 행들은 모두 'other'로 표시
    if target_race != 'other':
        X2.loc[~mask_target, 'other'] = 1.0
    else:
        mask_non_other = X[['white','black','asian']].any(axis=1)
        X2.loc[mask_non_other, 'other'] = 0.0
        X2.loc[~mask_non_other, 'other'] = 1.0

    return X2

def sum_race_shaps(shap_vals):
    
    all_races = [white,black,asian,other]
    all_race_idx = [list(X.columns).index(race) for race in all_races]

    # Step 1: Compute the sum of specified columns for each row
    new_column =shap_vals[:, all_race_idx].sum(axis=1)
    
    # Step 2: Add the new column to the array
    shap_vals = np.hstack((shap_vals, new_column.reshape(-1, 1)))
    
    # Step 3: Delete the specified columns
    shap_vals = np.delete(shap_vals, all_race_idx, axis=1)
    return shap_vals
def get_ranks(shap_vals):

    sorted_indices = np.argsort(-np.abs(shap_vals), axis=1)  # Indices of absolute values in descending order
    rank = np.empty_like(sorted_indices)             # Create an empty array of the same shape
    rows, cols = shap_vals.shape                            # Get the shape of the array
    rank[np.arange(rows)[:, None], sorted_indices] = np.arange(1, cols + 1)  # Assign ranks row-wise

    target_rank = rank[:,-1]

    return rank,target_rank

def compute_shap(X_train,X_test,Y_train,Y_test, seed):

    print('**********START**********')
    # Train model on new data
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],  # Number of boosting rounds
        'classifier__max_depth': [3, 5, 7,9,11],          # Maximum tree depth
        'classifier__learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage 
        'classifier__colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns for each tree
        'classifier__gamma': [0, 0.1, 0.2],          # Minimum loss reduction for a split
    }

    model = xgb.XGBClassifier(random_state=seed)
    grid_search = GridSearchCV(
        model, 
        param_grid,              # 3-fold cross-validation
        scoring='f1',   # Evaluation metric
        n_jobs=-1,            # Use all processors
        verbose=1             # Print progress
    )

    grid_search.fit(X_train, Y_train)
    
    # Extract the best model
    best_model = grid_search.best_estimator_

    explainer = GPUTreeExplainer(best_model,X_train,feature_perturbation = 'interventional')
    shap_values = explainer(X_test)
    pred = best_model.predict(X_test)
    return shap_values


def compute_fidelity(pred, sv, base):

    sv_sums = expit(np.sum(sv, axis=1)+base)
    binary_predictions = (sv_sums > 0.5).astype(float)
    fidelity = np.mean(binary_predictions == pred)
    match_idx = np.where(binary_predictions == pred)[0]
        
    return fidelity,match_idx
            

## Train the model with plain Age

In [120]:
base_shap_vals = list()
base_preds = list()
base_accs = list() 
base_f1s = list()
base_ranks = list()
base_race_ranks = list()

base_firsts = list()
base_percentages = list()
base_first_race_shap_vals = list()

base_models = list()

for seed in tqdm(seeds):
    np.random.seed(seed)
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    for train_val_idx, test_idx in splitter.split(X, Y):
        X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
        Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
    
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
        X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
        Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx]
    param_grid = {
        'classifier__n_estimators': [50, 100, 200],  # Number of boosting rounds
        'classifier__max_depth': [3, 5, 7,9,11],          # Maximum tree depth
        'classifier__learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage 
        'classifier__colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns for each tree
        'classifier__gamma': [0, 0.1, 0.2],          # Minimum loss reduction for a split
    }
    model = xgb.XGBClassifier(random_state=seed)
    grid_search = GridSearchCV(
        model, 
        param_grid,              # 3-fold cross-validation
        scoring='f1',   # Evaluation metric
        n_jobs=13,            # Use all processors
        verbose=1             # Print progress
    )

    grid_search.fit(X_train, Y_train)
    
    # Extract the best model
    best_model = grid_search.best_estimator_
    base_models.append(best_model)
    explainer = GPUTreeExplainer(best_model,X_train, feature_perturbation='interventional') 
    shap_values = explainer(X_test)
    
    sv = sum_race_shaps(shap_values.values)
    race_sv = sv[:,-1]
    base_rank,race_rank= get_ranks(sv)
    base_ranks.append(base_rank)
    
    pred = best_model.predict(X_test)
    # base_models.append(best_model)
    base_shap_vals.append(race_sv)
    base_preds.append(pred)
    base_accs.append(accuracy_score(Y_test,pred)*100)
    base_f1s.append(f1_score(Y_test,pred)*100)
    base_ranks.append(base_rank)
    base_race_ranks.append(race_rank)
    # First

    ## Indices of first
    first = [int(j) for j,v in enumerate(race_rank) if v == 1]
    base_firsts.append(first)
    base_percentages.append(len(first)/len(X_test) *100)
    base_first_race_shap_vals.append(race_sv[first])
print(f'Overall average acc: {sum(base_accs)/len(base_accs):.2f} average f1s : {sum(base_f1s)/len(base_f1s):.2f}')





  0%|                                                       | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 20%|█████████▍                                     | 1/5 [00:21<01:24, 21.03s/it]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 40%|██████████████████▊                            | 2/5 [00:38<00:56, 18.81s/it]

...

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


 80%|█████████████████████████████████████▌         | 4/5 [01:12<00:17, 17.68s/it]

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


100%|███████████████████████████████████████████████| 5/5 [01:29<00:00, 17.96s/it]

Overall average acc: 79.47 average f1s : 76.31


In [123]:
race_rank

array([13, 19,  9, ..., 11,  3,  9])

In [122]:
import pickle
from pathlib import Path
path = './results'
if not os.path.exists(path):
   # Create a new directory because it does not exist
   os.makedirs(path)

# # save
with open(path + '/Bucket_OVA_Income_base_shap_vals_cv.pickle', 'wb') as f:
    pickle.dump(base_shap_vals, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_base_accs_cv.pickle', 'wb') as f:
    pickle.dump(base_accs, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_base_f1s_cv.pickle', 'wb') as f:
    pickle.dump(base_f1s, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_base_age_ranks_cv.pickle', 'wb') as f:
    pickle.dump(base_race_ranks, f, pickle.HIGHEST_PROTOCOL)

with open(path + '/Bucket_OVA_Income_base_firsts_cv.pickle', 'wb') as f:
    pickle.dump(base_firsts, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_base_percentages_cv.pickle', 'wb') as f:
    pickle.dump(base_percentages, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_base_first_race_shap_vals_cv.pickle', 'wb') as f:
    pickle.dump(base_first_race_shap_vals, f, pickle.HIGHEST_PROTOCOL)


In [133]:
base_first_race_shap_vals

5

In [45]:
# base_ranks = list()
# base_avg_shaps = list()
# base_top_count = list()
# base_top_avg_shaps = list()
# for sv in base_result['shap_vals']:
#     sv = sv.values
#     base_rank= analyze_shap(sv)
#     base_ranks.append(base_rank)
#     age_rank = base_rank[:,-1]
#     base_first = [int(j) for j in range(len(age_rank)) if int(age_rank[j]) == 1]
#     base_first_count = len(base_first)
#     sv = sum_race_shaps(sv)
#     base_avg_shaps.append(np.mean(np.abs(sv[:,-1])))
    
#     base_top_avg_shaps.append(np.mean(np.abs(sv[base_first,-1])))
#     base_top_count.append(base_first_count)

# base_avg_shaps_m = np.mean(base_avg_shaps)
# base_top_count_m = np.mean(base_top_count)
# base_top_avg_shaps_m = np.mean(base_top_avg_shaps)

# base_avg_shaps_s = np.std(base_avg_shaps)
# base_top_count_s = np.std(base_top_count)
# base_top_avg_shaps_s = np.std(base_top_avg_shaps)

 # Race Bucetkization

In [127]:
bucket_fids = list()
bucket_shap_vals = list()
bucket_race_ranks = list()
bucket_firsts = list()
bucket_percentages = list()
bucket_first_race_shap_vals = list()


for race in all_races:
    # compute bin edges as shap values for bucketized data
    b_fids = list()
    b_shap_vals = list()
    b_race_ranks = list()
    b_firsts = list()
    b_percentages = list()
    
    b_first_race_shap_vals = list()


    X2 = assign_race(X, race)
    for i, seed in enumerate(seeds):
        np.random.seed(seed)
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
        for train_val_idx, test_idx in splitter.split(X2, Y):
            X_train_val, X_test = X2.iloc[train_val_idx], X2.iloc[test_idx]
            Y_train_val, Y_test = Y.iloc[train_val_idx], Y.iloc[test_idx]
        
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
        for train_idx, val_idx in splitter.split(X_train_val, Y_train_val):
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            Y_train, Y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx] 

        b_shap = compute_shap(X_train,X_test,Y_train,Y_test,seed)
        sv = sum_race_shaps(b_shap.values)
        race_sv = sv[:,-1]
        b_rank, b_race_rank = get_ranks(sv)

        preds = base_preds[i]
        base = b_shap.base_values
        b_fid, b_agreed = compute_fidelity(preds,sv,base)


        b_fids.append(b_fid)
        b_race_ranks.append(b_race_rank)
        b_shap_vals.append(race_sv)
        
        # First

        first = [int(j) for j,v in enumerate(b_race_rank) if v == 1]
        
        b_firsts.append(first)
        b_percentages.append(len(first)/len(X_test)*100 )
        b_first_race_shap_vals.append(race_sv[first])
    
    bucket_fids.append(b_fids)
    bucket_shap_vals.append(b_shap_vals)
    bucket_race_ranks.append(b_race_ranks)
    bucket_firsts.append(b_firsts)
    bucket_percentages.append(b_percentages)
    bucket_first_race_shap_vals.append(b_first_race_shap_vals)


  

**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********START**********
Fitting 5 folds for each of 270 candidates, totalling 1350 fits
**********

In [138]:
b_first_race_shap_vals

[array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([-0.77549565, -0.85126317]),
 array([], dtype=float64)]

In [129]:
with open(path + '/Bucket_OVA_Income_bucket_fids_cv.pickle', 'wb') as f:
    pickle.dump(bucket_fids, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_bucket_shap_vals_cv.pickle', 'wb') as f:
    pickle.dump(bucket_shap_vals, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_bucket_race_ranks_cv.pickle', 'wb') as f:
    pickle.dump(bucket_race_ranks, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_bucket_firsts_cv.pickle', 'wb') as f:
    pickle.dump(bucket_firsts, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_bucket_percentages_cv.pickle', 'wb') as f:
    pickle.dump(bucket_percentages, f, pickle.HIGHEST_PROTOCOL)
with open(path + '/Bucket_OVA_Income_bucket_first_race_shap_vals_cv.pickle', 'wb') as f:
    pickle.dump(bucket_first_race_shap_vals, f, pickle.HIGHEST_PROTOCOL)


In [98]:
# ew_bucket_downs = list()
# ew_bucket_avg_ranks = list()
# ew_bucket_top_avg_ranks = list()
# ew_bucket_top_counts = list()


# ew_bucket_fids = list()

# for race in all_races:
#     # compute bin edges as shap values for bucketized data

#     cut5_result = compute_shap(X,Y,base_result['models'],race)
#     cut5_ranks = list()
#     race_ranks = list()
#     for i,sv in enumerate(cut5_result['shap_vals']):
#         sv = sv.values
#         cut5_rank = analyze_shap(sv,plot=False)
#         cut5_ranks.append(cut5_rank)
#         race_rank = np.mean(cut5_rank[:,-1])
#         race_ranks.append(race_rank)
#     # Compute fidelity and indices where explanation is the same.
#     cut5_fidelities = list()
#     cut5_agreeds = list()
#     for i in range(FOLDS):
#         preds = base_result['preds'][i]
#         sv = cut5_result['shap_vals'][i].values
#         base = cut5_result['shap_vals'][i].base_values
#         cut5_fidelity,cut5_agreed = compute_fidelity(preds,sv,base)
#         cut5_fidelities.append(cut5_fidelity)
#         cut5_agreeds.append(cut5_agreed)
#     avg_fid = np.mean(cut5_fidelities)
#     avg_rank = np.mean(race_ranks)


#     # Compare shap values, rank differences and kendalls
#     shap_difs = list()
#     rank_difs = list()
#     kendalls = list()
#     avg_shaps = list()
#     for i in range(FOLDS):
#         s1 = sum_race_shaps(base_result['shap_vals'][i])
#         s2 = sum_race_shaps(cut5_result['shap_vals'][i])
#         _,_,rank_dif,_ = compare_results(s1, s2, base_ranks[i], race_ranks[i] ,cut5_agreeds[i])
#         rank_difs.append(rank_dif)
        

    
#     ew_bucket_fids.append(avg_fid)
#     ew_bucket_avg_ranks.append(avg_rank)


#     first_rank_shifts,base_first = get_first_rank_shift(base_ranks, rank_difs)
#     first_down = down_percent(first_rank_shifts)

#     ew_top_counts.append(np.mean([len(b) for b in base_first]))

#     # Get top shap vals
#     top_avg_shaps = list()
#     for i in range(FOLDS):
#         sv = sum_race_shaps(cut5_result['shap_vals'][i].values)
        
#         first_sv = abs(sv[base_first[i]])
#         cut5_avg_shap = np.mean(sv[:,-1])
#         top_avg_shaps.append(np.mean(first_sv))
#     ed_top_avg_shaps.append(top_avg_shaps)
    

In [99]:
cut5_avg_ranks = list()
attack_avg_ranks = list()
attack_avg_fidelities = list()
base_rank = np.mean(base_ranks[0][:,-1])
print(f'base rank {base_rank:.2f}')
for i,v in enumerate(ew_bucket_avg_ranks):
    if v:
        cur_avg_rank = v
        cur_fidelity = float(ew_bucket_fids[i])
        

        print(f'Attack rank: {cur_avg_rank:.2f}  new fid: {cur_fidelity:.4f}')

    
    attack_avg_ranks.append(cur_avg_rank)
    attack_avg_fidelities.append(cur_fidelity)
attack_avg_ranks = [abs(r) for r in attack_avg_ranks if r != None]

base rank 7.46


NameError: name 'ew_bucket_avg_ranks' is not defined

In [ ]:
ew_bucket_avg_ranks

In [ ]:
import mlresearch
mlresearch.utils.set_matplotlib_style()

In [ ]:
with open('Race_Income_base_rank_OVA.pickle', 'wb') as f:
    pickle.dump(base_rank, f, pickle.HIGHEST_PROTOCOL)
with open('Race_Income_avg_ranks_OVA.pickle', 'wb') as f:
    pickle.dump(attack_avg_ranks, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
x_values = ['Base']+ all_races
# Bar width and positions
bar_width = 0.4
x_indices = np.arange(len(x_values))

# Plot bars
plt.bar(x_indices,[base_rank] + attack_avg_ranks, width=bar_width, color='blue')

# Labeling and styling
plt.xticks(ticks=x_indices, labels=x_values)
plt.xlabel('Race Buckets')
plt.ylabel('Average Ranks')
plt.title('ACS Income Ranks')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display the plot
plt.tight_layout()
plt.show()